# Ah shit
## Here we go again

In [1]:
import markovify

Создал приложение в вк и достал из паблика "пацанские цитаты и союзмультфильм" 2000 постов. С тем, что цитаты иногда повторяются, я ничего не сделал. Как и с тем, что есть посты-нецитаты.

Есть еще один прикол: корпус маленький очень. Как его пополнять -- хз. Нет нормальной датабазы пацанских цитат.

In [47]:
import requests

In [58]:
# это будущие тексты цитат
pacan_quotes = []

for i in range(20):
    data = requests.get(
        'https://api.vk.com/method/wall.get', 
        params={
            "owner_id": -157341954,
            "offset": 100*i,
            "count": 100,
            "v":"5.92",
            "access_token": TOKEN # хрен покажу
        }
    ).json()
    # это я пополняю список текстами постов
    pacan_quotes.extend(item['text'] for item in data['response']['items'])

В итоге, без пустых постов, всего их 1953.

In [61]:
len([text for text in pacan_quotes if text])

1953

Markovify учится на тексте, так что соединяем цитаты в один текст и строим модель.

Проблема, связанная с этим: модель будет думать, что следующие друг за другом цитаты -- это норма языка.

In [755]:
corpus = ".\n".join([quote.lower() for quote in pacan_quotes if quote]).replace(' \n', '.\n')

In [756]:
text_model = markovify.Text(corpus, state_size=3)
model_json = text_model.to_json()
model = markovify.Text.from_json(model_json)

Получается что-нибудь такое, но не всегда. Иногда тупо две цитаты, склееные вместе. Потому что корпус маленький. Нет на свете стольких пацанских цитат, к сожалению(

In [757]:
model.make_short_sentence(1000)

'когдa-нибудь я напишу в статусе..cчастлив...закину парy фоток с сыном и больше никогда не буду колоться не люблю парфюм от lacoste.'

Мигабай, что ты делаешь, ахахах, зачем тебе диалоги из двух томов Войны и Мира?

Если совсем честно, они у меня с прошлого проекта валялись, а весь ВиМ закачивать было влом, так что вот.

Я сделал модель по диалогам из ВиМ.

In [600]:
corpus_tolstoy = open("vim-dialogue.txt").read().lower()

In [601]:
text_model_tolstoy = markovify.Text(corpus_tolstoy, state_size=3)
model_tolstoy_json = text_model_tolstoy.to_json()
model_tolstoy = markovify.Text.from_json(model_tolstoy_json)

Скомбайнил две модели (так, чтобы пацанские цитаты имели больший вес (10/1)).

In [602]:
model_with_tolstoy = markovify.combine([model, model_tolstoy], [10, 1])

ОЧЕНЬ редко получается комбинация двух жанров. Обычно либо ВиМ, либо цитаты. Иногда просто выпадает None.

In [805]:
model_with_tolstoy.make_short_sentence(1000)

'казаки видели, как быстро задергались его руки и ноги, несмотря на то, что это так быстро совершилось, доказывало то, что это означало, его смерть…до слез....'

*Миша Сонькин*